In [19]:
import numpy as np
import pandas as pd
import requests
import yfinance as yf
import prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import seaborn as sns
import matplotlib.pyplot as plt
import prophet.plot as ppl
import datetime as dt
import itertools

### Start Date and Today

In [20]:
from datetime import date

start_date = "2017-01-01"
today = date.today().strftime("%Y-%m-%d")

### Load the Data

In [21]:
ticker = ["GOOG"]
data = yf.download(ticker, start_date, today)

[*********************100%***********************]  1 of 1 completed


### Data Manipulation of Stock Data

In [22]:
data.reset_index(inplace=True)
data["Date"] = data["Date"].dt.tz_localize(None)

In [23]:
data.head(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1528 non-null   datetime64[ns]
 1   Open       1528 non-null   float64       
 2   High       1528 non-null   float64       
 3   Low        1528 non-null   float64       
 4   Close      1528 non-null   float64       
 5   Adj Close  1528 non-null   float64       
 6   Volume     1528 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 83.7 KB


(        Date       Open       High        Low      Close  Adj Close    Volume
 0 2017-01-03  38.940498  39.481499  38.790001  39.306999  39.306999  33146000
 1 2017-01-04  39.417999  39.567001  39.158001  39.345001  39.345001  21460000
 2 2017-01-05  39.304001  39.723999  39.250999  39.701000  39.701000  26704000
 3 2017-01-06  39.763000  40.395000  39.610199  40.307499  40.307499  32804000
 4 2017-01-09  40.320000  40.498299  40.141499  40.332500  40.332500  25492000,
 None)

In [24]:
df_train = data[["Date", "Close"]]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

In [25]:
df_train

,ds,y
0,2017-01-03,39.306999
1,2017-01-04,39.345001
2,2017-01-05,39.701000
3,2017-01-06,40.307499
4,2017-01-09,40.332500
...,...,...
1523,2023-01-23,101.209999
1524,2023-01-24,99.209999
1525,2023-01-25,96.730003
1526,2023-01-26,99.160004


### Model

In [26]:
baseline_model = Prophet()
baseline_model.fit(df_train)

12:49:26 - cmdstanpy - INFO - Chain [1] start processing
12:49:27 - cmdstanpy - INFO - Chain [1] done processing


### Hyperparameter Tuning

In [27]:
# Some code here

### Save Model

In [28]:
# from prophet.serialize import model_to_json, model_from_json
#
# with open('serialized_model.json', 'w') as fout:
#     fout.write(model_to_json(auto_model))  # Save model
#
# with open('serialized_model.json', 'r') as fin:
#     auto_model = model_from_json(fin.read())  # Load model

### Predict Future

In [29]:
period = 365 * 2
preds = baseline_model.predict(df_train)
future = baseline_model.make_future_dataframe(periods=period)
forecast = baseline_model.predict(future)

In [30]:
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
2253,2025-01-22,-9.792065,-68.089937,49.523348,-70.605443,48.202486,1.901344,1.901344,1.901344,-0.095000,-0.095000,-0.095000,1.996344,1.996344,1.996344,0.0,0.0,0.0,-7.890721
2254,2025-01-23,-9.926442,-67.679978,51.506995,-70.921418,48.289407,2.007625,2.007625,2.007625,-0.091877,-0.091877,-0.091877,2.099501,2.099501,2.099501,0.0,0.0,0.0,-7.918817
2255,2025-01-24,-10.060819,-68.448077,50.217127,-71.237393,48.293962,2.107432,2.107432,2.107432,-0.092426,-0.092426,-0.092426,2.199858,2.199858,2.199858,0.0,0.0,0.0,-7.953387
2256,2025-01-25,-10.195197,-68.022194,49.888873,-71.553336,48.283026,2.674051,2.674051,2.674051,0.377079,0.377079,0.377079,2.296971,2.296971,2.296971,0.0,0.0,0.0,-7.521146
2257,2025-01-26,-10.329574,-70.788103,51.336915,-71.867447,48.272091,2.767457,2.767457,2.767457,0.377079,0.377079,0.377079,2.390377,2.390377,2.390377,0.0,0.0,0.0,-7.562118


### Plot

In [31]:
import plotly.offline as py

fig = ppl.plot_plotly(baseline_model, forecast)
py.iplot(fig)